In [19]:
import os
import cv2
import numpy as np
import csv

# Video, output csv, output dir path
video_path = 'data/videos/test9_cs412-21s.mp4'
output_csv = 'output/txt/annotations.csv'
output_dir = "output/image/maskSeg"

if not os.path.exists(video_path):
        print(f"{video_path}: File not found.")

if not os.path.exists(output_dir):
        os.makedirs(output_dir)


# Load video
cap = cv2.VideoCapture(video_path)
sampling_frequency = 1  # seconds between samples

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
ret, frame = cap.read()
height, width, channels = frame.shape

# frames Numpy array 
frames = []

# Extracting frames, chronologically forwards
count = 0
frame_index = 0
while frame_index <= cap.get(cv2.CAP_PROP_FRAME_COUNT):
    ret, frame = cap.read()
    if not ret:
        break  # End of video
    frames.append(frame)
    
    # Increment frame_index
    count += 1
    cap.set(cv2.CAP_PROP_POS_FRAMES, int((count * fps) / sampling_frequency))

cap.release()

# Convert to NumPy array
frames_array = np.array(frames)

# Assuming white background
background_color = [253, 253, 253] 

# Removing background pixels, creating masks
null_color = [50, 205, 50]
for frame in frames_array:
    for row in range(height):
        for col in range(width):
            if (frame[row][col] == background_color).all():
                frame[row][col] = null_color # might add a transparency channel instead

# Subtracting adjacent frames
for index in range(count - 1): 
    curr_frame = frames_array[index]
    next_frame = frames_array[index + 1]
    for row in range(height):
        for col in range(width): 
            # if ((next_frame[row][col] == null_color).all() or (curr_frame[row][col] == null_color).all()):
            #     continue
            if ((curr_frame[row][col] == next_frame[row][col]).all()):
                curr_frame[row][col] = null_color 

# Checking emptiness of frame
frame_pixel_count = [0] * count 
for index in range(count): 
    curr_frame = frames_array[index]
    for row in range(height):
        for col in range(width): 
            if ((curr_frame[row][col] == null_color).all()):
                continue
            frame_pixel_count[index] += 1

# Output to csv
with open(output_csv, 'a', newline='') as csvfile:
    fieldnames = ['index', 'frame', 'value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    index = 0
    for value in frame_pixel_count:
        writer.writerow({'index' : index, 'frame' : int((index * fps) / sampling_frequency), 'value' : value})
        index += 1

# frames_array to image for visualization
index = 0
for frame in frames_array:
    img_file = os.path.join(output_dir, f"scene_{index}.jpg")
    cv2.imwrite(img_file, frame)
    index += 1
            